In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_890530/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_45_0,0.999888,0.808638,0.818942,0.999840,0.000484,1.279638,0.647633,2.179800e-04,0.009724,0.022010,1.007161,0.022357,145.265024,286.600197,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_48_2,0.999888,0.808638,0.818942,0.999627,0.000484,1.279638,0.647633,1.648935e-03,0.009724,0.022010,1.007161,0.022357,145.265024,286.600197,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_38_1,0.999888,0.808638,0.818942,1.000000,0.000484,1.279638,0.647633,3.886705e-10,0.009724,0.022010,1.007161,0.022357,145.265024,286.600197,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_38_2,0.999888,0.808638,0.818942,1.000000,0.000484,1.279638,0.647633,3.886705e-10,0.009724,0.022010,1.007161,0.022357,145.265024,286.600197,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_38_3,0.999888,0.808638,0.818942,1.000000,0.000484,1.279638,0.647633,3.886705e-10,0.009724,0.022010,1.007161,0.022357,145.265024,286.600197,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,model_1_3_1,0.923136,0.724190,0.769378,0.908218,0.332802,1.844342,0.824920,2.735300e-01,0.699905,0.576890,5.919303,0.585976,132.200413,273.535586,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
469,model_1_3_0,0.922352,0.724117,0.766109,0.907131,0.336198,1.844831,0.836614,2.767692e-01,0.708457,0.579825,5.969489,0.588957,132.180112,273.515285,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
470,model_1_2_2,0.908765,0.712670,0.756884,0.918453,0.395024,1.921376,0.869612,5.205241e-01,0.770175,0.628509,6.839022,0.638407,131.857620,273.192793,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
471,model_1_2_1,0.899823,0.707963,0.732810,0.916671,0.433741,1.952851,0.955722,5.318971e-01,0.851029,0.658590,7.411322,0.668962,131.670616,273.005788,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
